In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from xgboost import XGBRegressor

import multiprocessing as mp
from functools import partial
from helper_functions import *

## Load Data

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv').sort_values(by=['cfips', 'first_day_of_month']).reset_index().drop('index', axis=1)
results_data = pd.read_csv('all_results_with_category.csv')

In [3]:
test

,row_id,cfips,first_day_of_month
0,1001_2022-11-01,1001,2022-11-01
1,1001_2022-12-01,1001,2022-12-01
2,1001_2023-01-01,1001,2023-01-01
3,1001_2023-02-01,1001,2023-02-01
4,1001_2023-03-01,1001,2023-03-01
...,...,...,...
25075,56045_2023-02-01,56045,2023-02-01
25076,56045_2023-03-01,56045,2023-03-01
25077,56045_2023-04-01,56045,2023-04-01
25078,56045_2023-05-01,56045,2023-05-01


In [4]:
results_data.head()

,Country,LSTM_W9_H8,SV_MAPE_x,LR_MAPE,XGBR_MAPE,tf_dense_MAPE,dense_W9_H8_MV,category
0,1001,0.732748,1.835232,0.992271,2.257356,1.504614,0.724775,Dense_MV
1,1003,0.747286,5.230218,4.317589,93.943110,4.143341,2.480568,LSTM
2,1005,1.273616,1.101416,1.833943,0.912357,1.395184,4.606141,XGBR
3,1007,9.315642,3.574082,4.259990,4.514549,5.228748,24.276476,SV
4,1009,0.766072,2.116541,1.285277,2.398140,1.442244,1.445454,LSTM
5,1011,4.382242,7.463520,10.230007,8.838513,7.461480,7.102029,LSTM
6,1013,2.639335,6.244084,5.244693,5.939532,8.958930,27.140242,LSTM
7,1015,2.105894,1.793359,2.107688,1.878761,3.481473,1.868864,SV
8,1017,6.220597,3.315358,3.400152,4.239078,6.806362,14.319926,SV
9,1019,2.760927,13.815006,15.852276,8.066227,7.121981,4.013296,LSTM


In [5]:
%%time
if __name__ == '__main__':
    HORIZON = 8
    WINDOW_SIZE = 9

    train = pd.read_csv('train.csv')
    IDS = train.cfips.unique()
    
    sample_ids = IDS[:50]
    
    window_size = WINDOW_SIZE
    horizon = HORIZON
    epoch = 500
    split_size = 0
    
    cfips_list = []
    forecasts = []

    pool = mp.Pool(16)

    func = partial(get_score_for_competition, train, results_data, window_size, horizon, epoch, split_size)
    results = pool.map(func, IDS)
    pool.close()
    pool.join()
    
    for i in range(len(results)):
        c = results[i][0]
        cfips_list.append(c)
        forecasts.append(results[i][1])

CPU times: total: 4.12 s
Wall time: 1h 15min 30s


In [6]:
test['microbusiness_density'] = np.array(forecasts).reshape((-1))
submission = test[['row_id', 'microbusiness_density']]

In [7]:
submission.head(10)

,row_id,microbusiness_density
0,1001_2022-11-01,3.169709
1,1001_2022-12-01,3.198731
2,1001_2023-01-01,3.213416
3,1001_2023-02-01,3.214808
4,1001_2023-03-01,3.208710
5,1001_2023-04-01,3.219334
6,1001_2023-05-01,3.234708
7,1001_2023-06-01,3.239502
8,1003_2022-11-01,8.237763
9,1003_2022-12-01,8.221723


In [8]:
submission.to_csv('submission_3.csv', index=False)